In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# From https://github.com/kratzert/pangeo_lstm_example/blob/master/LSTM_for_rainfall_runoff_modelling.ipynb
def calc_nse(obs: np.array, sim: np.array) -> float:
    """Calculate Nash-Sutcliff-Efficiency.

    :param obs: Array containing the observations
    :param sim: Array containing the simulations
    :return: NSE value.
    """
    # only consider time steps, where observations are available
    sim = np.delete(sim, np.argwhere(obs < 0), axis=0)
    obs = np.delete(obs, np.argwhere(obs < 0), axis=0)

    # check for NaNs in observations
    sim = np.delete(sim, np.argwhere(np.isnan(obs)), axis=0)
    obs = np.delete(obs, np.argwhere(np.isnan(obs)), axis=0)

    denominator = np.sum((obs - np.mean(obs)) ** 2)
    numerator = np.sum((sim - obs) ** 2)
    nse_val = 1 - numerator / denominator

    return nse_val

In [ ]:
# Read all input txt files
import glob

path = "../Input files (.txt)"
all_files = glob.glob(path + "/*.txt")

df_dict = {}
for file_path in all_files:
    print(f"Reading file: {file_path}")
    # Name is formatted `./Input files (.txt)/nve_inp_XX.txt`
    number = int(file_path.split("_")[-1].split(".")[0])

    df = pd.read_csv(
        file_path,
        encoding="cp1252",
        skiprows=[0],
        delimiter=r"\s+",
        parse_dates=[["dd.mm.yyyy", "hh:mm:ss"]],
    )
    df = df.rename(columns={"dd.mm.yyyy_hh:mm:ss": "timestamp"})
    df_dict[number] = df

In [ ]:
# All files have equal values for grC and grC.1
for df in df_dict.values():
    print(df["grC"].equals(df["grC.1"]))

In [ ]:
print(df_dict[1].keys())
n_without_gaps = 0
without_gaps = {}
for n in df_dict:
    df = df_dict[n]
    df1 = (
        df.sort_values("timestamp")
        .apply(lambda x: x.diff().max())
        .reset_index(name="max_diff")
    )
    display(df1)
    if df1["max_diff"][0].days < 2:
        print("Without gap")
        n_without_gaps += 1
        without_gaps[n] = df

print(f"Number of timeseries without gaps: {n_without_gaps}")

In [ ]:
long_without_gaps = {}
# Catchments without measurement gaps
for id in sorted(without_gaps.keys()):
    df = without_gaps[id]
    # print(df)
    _min = df["timestamp"].min()
    _max = df["timestamp"].max()
    _diff = _max - _min
    print(f"{id:3d} {_min} {_max} {_diff}")
    if _diff.days >= 5843:
        long_without_gaps[id] = without_gaps[id]
        # print('Added catchment')

print(f"\nNumber of catchment with long measurement period: {len(long_without_gaps)}")
print(f"Ids: {list(long_without_gaps.keys())}")

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def plot_multi_histogram(catchments):
    # plot with various axes scales

    plt.figure(figsize=(5, 5))

    all_nses = []
    max_y = None
    for offset in range(12):
        plt.subplot(4, 4, offset + 1)
        offset += 1
        nses = []
        all_nses.append(nses)
        # Calculate nse based on model which predicts the previous day for all timeseries without gaps
        for id in sorted(catchments.keys()):
            runoff = catchments[id]["m3/s"].to_numpy()
            offset = offset
            nse = calc_nse(runoff[offset:], runoff[:-offset])
            nses.append(nse)
            # print(f'{id:02d}, nse: {nse:.3f}')

        # the histogram of the data
        n, bins, patches = plt.hist(
            all_nses[-1],
            bins=[0, 0.10, 0.20, 0.30, 0.40, 0.50, 0.6, 0.7, 0.8, 0.9, 1.0],
        )
        plt.grid(False)
        plt.xticks([])
        plt.yticks([])
        plt.xlabel(f"{offset} days")
        if max_y is None:
            max_y = max(n)
        plt.ylim(top=max_y, bottom=0)

    plt.show()

In [ ]:
plot_multi_histogram(without_gaps)
plot_multi_histogram(df_dict)

In [ ]:
def create_test_and_validation_set(
    df_dict, split_date, length, x_parameters=["mm", "grC"], y_parameters=["m3/s"]
):
    TRAIN_X_DICT = {}
    TRAIN_Y_DICT = {}
    TEST_X_DICT = {}
    TEST_Y_DICT = {}
    for i, _id in enumerate(df_dict):
        # if i % 2 == 0:
        #     print(f'{i+1}/{len(df_dict)}: Parsing catchment {_id}')
        catchment_dataframe = df_dict[_id]
        train_dataframe = catchment_dataframe[
            catchment_dataframe["timestamp"] <= split_date
        ]
        test_dataframe = catchment_dataframe[
            catchment_dataframe["timestamp"] > split_date
        ]

        TRAIN_X_DICT[_id] = substring_array(
            train_dataframe[x_parameters].to_numpy()[:-1], length
        )
        TRAIN_Y_DICT[_id] = train_dataframe[y_parameters].to_numpy()[length:]

        TEST_X_DICT[_id] = substring_array(
            test_dataframe[x_parameters].to_numpy()[:-1], length
        )
        TEST_Y_DICT[_id] = test_dataframe[y_parameters].to_numpy()[length:]

        assert len(TRAIN_X_DICT[_id]) == len(TRAIN_Y_DICT[_id])
        assert len(TEST_X_DICT[_id]) == len(TEST_Y_DICT[_id])

    return TRAIN_X_DICT, TRAIN_Y_DICT, TEST_X_DICT, TEST_Y_DICT


def substring_array(array, length):
    python_list = array.tolist()
    new_list = []
    for i in range(len(python_list) - length + 1):
        new_list.append(python_list[i : i + length])
    return np.array(new_list)

In [ ]:
import datetime

TRAIN_X_DICT, TRAIN_Y_DICT, TEST_X_DICT, TEST_Y_DICT = create_test_and_validation_set(
    long_without_gaps,
    datetime.datetime(2007, 8, 31),
    1,
    x_parameters=["mm", "grC", "m3/s"],
)

In [ ]:
# Training using k_nearest_neighbour
from sklearn.neighbors import KNeighborsRegressor

stats_dict = {"desc": []}
lookback_days = [1, 2, 3, 7, 30, 100, 365]
x_parameters_list = [["mm", "grC"], ["mm", "grC", "m3/s"]]

for lookback in lookback_days:
    print(f"Number of days lookback: {length}")
    for use_rainoff_for_x in [False, True]:
        x_parameters = (
            x_parameters_list[1] if use_rainoff_for_x else x_parameters_list[0]
        )
        (
            TRAIN_X_DICT,
            TRAIN_Y_DICT,
            TEST_X_DICT,
            TEST_Y_DICT,
        ) = create_test_and_validation_set(
            long_without_gaps,
            datetime.datetime(2007, 8, 31),
            lookback,
            x_parameters=x_parameters,
        )

        for n_neighbors in [1, 3, 5, 7, 10]:
            stats_dict["desc"].append(
                f"lookback {lookback}, use_rainoff {use_rainoff_for_x}, n_neighbors {n_neighbors}"
            )
            for _id in TRAIN_X_DICT.keys():
                # print(_id)
                train_x, train_y, test_x, test_y = (
                    TRAIN_X_DICT[_id],
                    TRAIN_Y_DICT[_id],
                    TEST_X_DICT[_id],
                    TEST_Y_DICT[_id],
                )

                model = KNeighborsRegressor(n_neighbors=n_neighbors)
                train_x_flatend = train_x.reshape(
                    (train_x.shape[0], -1), order="F"
                )  # https://stackoverflow.com/a/37500847
                # print(train_x_flatend[0:2])
                # print(train_y[0])
                model.fit(train_x_flatend, train_y)

                test_x_flatend = test_x.reshape((test_x.shape[0], -1), order="F")
                pred_y = model.predict(test_x_flatend)
                nse = calc_nse(pred_y, test_y)
                print(f"Catchment {_id:2d}: {nse:.3f}")
                if _id not in stats_dict:
                    stats_dict[_id] = []
                stats_dict[_id].append(nse)

display(nse)

In [ ]:
display(stats_dict)

In [ ]:
stats_df = pd.DataFrame(data=stats_dict)

In [ ]:
stats_df.to_excel("knn.xlsx")